In [ ]:
# pip install pandas
# !pip install pycountry_convert

## Libraries

In [2]:
import pandas as pd 

In [ ]:
import pycountry_convert as pc

In [3]:
# ⚙️ Settings
pd.set_option('display.max_columns', None) # display all columns
import warnings
warnings.filterwarnings('ignore') # ignore warnings

### individual csv load

In [8]:
data = pd.read_csv('reg_tech_dt_y_es.csv')
data1 = pd.read_csv('country_tech_dt_grid_y_es.csv')
data2 = pd.read_csv('reg_indic_y_re_share.csv') 
data3 = pd.read_csv('pub_investment.csv')

#snake_case columns
data.columns = [column.lower().replace(" ", "_") for column in data.columns]
data1.columns = [column.lower().replace(" ", "_") for column in data1.columns]
data2.columns = [column.lower().replace(" ", "_") for column in data2.columns]
data3.columns = [column.lower().replace(" ", "_") for column in data3.columns]

In [21]:
df_region = pd.merge(data, data2, on=['region', 'year'])

# Cambiar la columna de technology a los mismos valores pq tb está la columna de on-grid y off-grid
df_country = pd.merge(data1, data3, on=['country/area', 'year'])

### remove columns with no info (" - ") or value 0.00  

In [ ]:
# df_country.eq("-").sum()
# df_region.eq("-").sum()

# df_country.eq("0.00").sum()
# df_region.eq("0.00").sum()

In [ ]:
df_country1 = df_country[(df_country['electricity_statistics_(mw/gwh)'] != '-') & (df_country['electricity_statistics_(mw/gwh)'] != '0.00') & (df_country['public_investments_(2021_million_usd)'] != 0.00)]
df_region1 = df_region[(df_region['electricity_statistics'] != '-') & (df_region['re_share_of_electricity_capacity_and_generation_%'] != '-') & (df_region['electricity_statistics'] != '0.00')]

# pass to csv
country_df = df_country1.to_csv('country_df.csv', index=False)
region_df = df_region1.to_csv('region_df.csv', index=False)

In [ ]:
# read combined csv
data_country = pd.read_csv('country_df.csv')
data_region = pd.read_csv('region_df.csv')

### copy

In [125]:
df_c = data_country.copy()
df_r = data_region.copy()

### clean

In [ ]:
df_c.rename(columns={'country/area': 'country'}, inplace=True)

In [ ]:
# clean � from coutry names
df_c['country'] = df_c['country'].replace({
    "R�union": "Réunion",
    "C�te d'Ivoire": "Côte d'Ivoire",
    "T�rkiye": "Türkiye"
})

# remove () from country names
df_c['country'] = df_c['country'].replace({
    "Bahamas (the)": "Bahamas",
    "Bolivia (Plurinational State of)": "Bolivia",
    "Central African Republic (the)": "Central African Republic",
    "Comoros (the)": "Comoros",
    "Congo (the)": "Congo",
    "Cook Islands (the)": "Cook Islands",
    "Democratic Republic of the Congo (the)": "Democratic Republic of the Congo",
    "Dominican Republic (the)": "Dominican Republic",
    "Gambia (the)": "Gambia",
    "Iran (Islamic Republic of)": "Iran",
    "Kosovo": "Republic of Kosovo",
    "Lao People's Democratic Republic (the)": "Lao People's Democratic Republic",
    "Marshall Islands (the)": "Marshall Islands",
    "Micronesia (Federated States of)": "Micronesia",
    "Netherlands (Kingdom of the)": "Netherlands",
    "Niger (the)": "Niger",
    "Philippines (the)": "Philippines",
    "Republic of Moldova (the)": "Republic of Moldova",
    "State of Palestine (the)": "Palestine",
    "Sudan (the)": "Sudan",
    "Syrian Arab Republic (the)": "Syria",
    "Timor-Leste": "East Timor",
    "United Republic of Tanzania (the)": "Tanzania",
    "United States of America (the)": "United States"
})

In [154]:
# function to get the region
def get_continent(country):
    try:
        country_code = pc.country_name_to_country_alpha2(country, cn_name_format="default")
        continent_code = pc.country_alpha2_to_continent_code(country_code)
        continent_name = {
            "AF": "Africa",
            "AS": "Asia",
            "EU": "Europe",
            "NA": "North America",
            "SA": "South America",
            "OC": "Oceania"
        }
        return continent_name[continent_code]
    except:
        return "Unknown"

In [171]:
# apply function to create col for continent
df_c['continent'] = df_c['country'].apply(get_continent)